In [1]:
import os.path
import numpy as np
import torch
import torchvision.utils as vutils
import neural_network as nn
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20,3)

device = torch.device("cuda:0")

def make_grid(a, b, steps):
    a, b = np.array(a), np.array(b)
    if steps > 1:
        difference = (b-a)/(steps-1)
        full_matrix = np.tile(a[np.newaxis,:], (steps, 1)) + np.tile(difference[np.newaxis,:], (steps, 1))*np.tile(np.arange(steps)[:, np.newaxis], (1, a.shape[0]))
    else:
        full_matrix = np.tile(a[np.newaxis,:], (steps, 1))
    return full_matrix

def combine_fixed_noise_with_classes(num_samples, labels_one_hot, nz):
    size = labels_one_hot.size()
    fixed_noise = torch.randn(num_samples, nz, 1, 1, device=device).repeat(size[0], 1, 1, 1)
    labels_one_hot = labels_one_hot.repeat(1,num_samples).view(size[0]*num_samples, size[1])
    return torch.cat([fixed_noise, labels_one_hot.unsqueeze(2).unsqueeze(3)], dim=1)

def get_networks(hack, cropsize, nclasses, load_path, load_epoch):
    netG, netD = nn.get_generator_and_discriminator(
        nc_arg=3, nz_arg=100, ngf_arg=64, ndf_arg=64, device_arg=device, hack_arg=hack, crop_size_arg=cropsize, num_classes_arg=nclasses
    )
    netG.load_state_dict(torch.load(os.path.join(load_path, f'netG_epoch_{load_epoch}.pth'), map_location=device))
    netD.load_state_dict(torch.load(os.path.join(load_path, f'netD_epoch_{load_epoch}.pth'), map_location=device))
    return netG, netD

def show_transition(a, b, steps, numsamples, netG, netD):
    netG.eval()
    grid = torch.FloatTensor(make_grid(a, b, steps)).to(device)
    input_for_generator = combine_fixed_noise_with_classes(numsamples, grid, 100)

    samples = netG(input_for_generator)
    show(vutils.make_grid(samples, normalize=True, nrow=8))

In [2]:
%matplotlib widget
def show(img):
    npimg = img.cpu().detach().numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')

classes {'amsterdam': 0, 'manhattan': 1, 'paris': 2, 'vienna': 3}
Classes: {'amsterdam': 0, 'dc': 1, 'florence': 2, 'las_vegas': 3, 'new_york': 4, 'paris': 5}

In [9]:
# DanielNet
numsamples = 1
hack = False
cropsize = 128
nclasses = 4
load_path = '/home/mbachl/repos/dl_visualcomputing2018/assignments/assignment3/runs/Jan24_14-09-59_gpu/'
load_epoch = 1900
netG, netD = get_networks(hack, cropsize, nclasses, load_path, load_epoch)

In [27]:
# MaxNet
numsamples = 1
hack = True
cropsize = 128
nclasses = 4
load_path = '/home/mbachl/repos/dl_visualcomputing2018/assignments/assignment3/runs/Jan25_17-14-50_gpu/'
load_epoch = 2100
netG, netD = get_networks(hack, cropsize, nclasses, load_path, load_epoch)

In [28]:
# 4 classes
plt.close()
np.random.seed(54)

amsterdam = np.array([1., 0, 0, 0])
manhattan = np.array([0., 1, 0, 0])
paris = np.array([0., 0, 1, 0])
vienna = np.array([0., 0, 0, 1])

steps = 5
show_transition(amsterdam, manhattan, steps, numsamples, netG, netD)

FigureCanvasNbAgg()

In [20]:
# MaxNet Stanford
numsamples = 1
hack = True
cropsize = 128
nclasses = 5
load_path = '/home/mbachl/repos/dl_visualcomputing2018/assignments/assignment3/runs/Feb04_19-31-33_gpu/'
load_epoch = 3100064
netG, netD = get_networks(hack, cropsize, nclasses, load_path, load_epoch)

In [25]:
# 5 classes
plt.close()
np.random.seed(54)

amsterdam = np.array([1., 0, 0, 0, 0])
dc = np.array([0., 1, 0, 0, 0])
florence = np.array([0., 0, 1, 0, 0])
las_vegas = np.array([0., 0, 0, 1, 0])
new_york = np.array([0., 0, 0, 0, 1])

steps = 5
show_transition(amsterdam, new_york, steps, numsamples, netG, netD)

FigureCanvasNbAgg()